In [1]:
import numpy
import io
from itertools import permutations

read training file and create 

In [2]:
def read_file_init_table(fname):
    tag_count = {}
    tag_count['<start>'] = 0
    word_tag = {}
    tag_trans = {}
    with open(fname) as f:
        content = f.readlines()
    # you may also want to remove whitespace characters like `\n` at the end of each line
    content = [x.strip() for x in content]
    idx_line = 0
    is_first_word = 0
    
    while idx_line < len(content):
        prev_tag = '<start>'
        while not content[idx_line].startswith('</kalimat'):
            if  not content[idx_line].startswith('<kalimat'):
                content_part = content[idx_line].split('\t')
                if content_part[1] in tag_count:
                    tag_count[content_part[1]] += 1
                else:
                    tag_count[content_part[1]] = 1
                    
                current_word_tag = content_part[0]+','+content_part[1]
                if current_word_tag in word_tag:
                    word_tag[current_word_tag] += 1
                else:    
                    word_tag[current_word_tag] = 1
                    
                if is_first_word == 1:
                    current_tag_trans = '<start>,'+content_part[1]
                    is_first_word = 0
                else:
                    current_tag_trans = prev_tag+','+content_part[1]
                    
                if current_tag_trans in tag_trans:
                    tag_trans[current_tag_trans] += 1
                else:
                    tag_trans[current_tag_trans] = 1                    
                prev_tag = content_part[1]   
                
            else:
                tag_count['<start>'] += 1
                is_first_word = 1
            idx_line = idx_line + 1

        idx_line = idx_line+1       
    return tag_count, word_tag, tag_trans

In [3]:
tag_count, word_tag, tag_trans = read_file_init_table('sample_postagged.txt')
print(tag_count)
print(word_tag)
print(tag_trans)

{'<start>': 10, 'NN': 43, 'SC': 9, 'VB': 30, 'NNP': 23, 'JJ': 14, 'RB': 3, 'IN': 10, 'Z': 15, 'CD': 7, 'CC': 6, 'PR': 3, 'PRP': 1, 'MD': 3, 'FW': 1}
{'Kera,NN': 1, 'untuk,SC': 4, 'amankan,VB': 1, 'pesta olahraga,NN': 1, 'Pemerintah,NNP': 1, 'kota,NNP': 2, 'Delhi,NNP': 3, 'mengerahkan,VB': 1, 'monyet,NN': 9, 'mengusir,VB': 3, 'monyet-monyet,NN': 2, 'lain,JJ': 2, 'yang,SC': 3, 'berbadan,VB': 2, 'lebih,RB': 1, 'kecil,JJ': 4, 'dari,IN': 2, 'arena,NN': 2, 'Pesta Olahraga,NNP': 1, 'Persemakmuran,NNP': 1, '.,Z': 8, 'Beberapa,CD': 1, 'laporan,NN': 1, 'menyebutkan,VB': 1, 'setidaknya,RB': 1, '10,CD': 2, 'ditempatkan,VB': 1, 'di,IN': 6, 'luar,NN': 1, 'lomba,NN': 1, 'dan,CC': 6, 'pertandingan,NN': 1, 'ibukota,NNP': 2, 'India,NNP': 2, 'Pemkot,NNP': 1, 'memiliki,VB': 2, '28,CD': 1, 'berencana,VB': 1, 'mendatangkan,VB': 1, 'sejenis,NN': 1, 'negara bagian,NNP': 1, 'Rajasthan,NNP': 1, 'Jumlah,NN': 1, 'mencapai,VB': 1, 'ribuan,CD': 1, ',,Z': 5, 'sebagian besar,NN': 1, 'berada,VB': 1, 'kantor-kantor,NN'

In [4]:
def create_trans_prob_table(tag_trans, tag_count):
    print(tag_trans)
    trans_prob = {}
    for tag1 in tag_count.keys():
        for tag2 in tag_count.keys():
            #print('tag1 = ')
            #print(tag1)
            trans_idx = tag1+','+tag2
            #print('trans_idx = ')
            #print(trans_idx)
            if trans_idx in tag_trans:
                #print(trans_idx)
                trans_prob[trans_idx] = tag_trans[trans_idx]/tag_count[tag1]
    return trans_prob

In [5]:
trans_prob = create_trans_prob_table(tag_trans, tag_count)
print(trans_prob)

{'<start>,NN': 5, 'NN,SC': 6, 'SC,VB': 7, 'VB,NN': 12, '<start>,NNP': 3, 'NNP,NNP': 13, 'NNP,VB': 4, 'NN,JJ': 9, 'JJ,SC': 2, 'VB,RB': 2, 'RB,JJ': 1, 'JJ,IN': 1, 'IN,NN': 4, 'NN,NNP': 2, 'NNP,Z': 5, '<start>,CD': 1, 'CD,NN': 6, 'NN,VB': 5, 'RB,CD': 1, 'VB,IN': 5, 'NN,NN': 9, 'NN,CC': 4, 'CC,NN': 3, 'NN,IN': 3, 'IN,NNP': 4, 'VB,CD': 3, 'CC,VB': 2, 'VB,VB': 2, 'CD,Z': 1, 'Z,NN': 3, 'NN,PR': 3, 'PR,VB': 2, 'JJ,Z': 5, 'VB,JJ': 3, 'Z,VB': 2, 'Z,CC': 1, 'JJ,CC': 1, 'VB,PRP': 1, 'PRP,MD': 1, 'MD,VB': 3, 'SC,NN': 1, 'JJ,JJ': 1, 'JJ,VB': 1, 'VB,Z': 1, 'CC,FW': 1, 'FW,VB': 1, 'SC,RB': 1, 'RB,VB': 1, 'NN,Z': 1, 'JJ,MD': 2, 'IN,CD': 2, '<start>,Z': 1, 'PR,SC': 1, 'Z,Z': 1, 'VB,NNP': 1, 'NNP,IN': 1}
{'<start>,NN': 0.5, '<start>,NNP': 0.3, '<start>,Z': 0.1, '<start>,CD': 0.1, 'NN,NN': 0.20930232558139536, 'NN,SC': 0.13953488372093023, 'NN,VB': 0.11627906976744186, 'NN,NNP': 0.046511627906976744, 'NN,JJ': 0.20930232558139536, 'NN,IN': 0.06976744186046512, 'NN,Z': 0.023255813953488372, 'NN,CC': 0.09302

In [6]:
def create_emission_prob_table(word_tag, tag_count):
    emission_prob = {}
    for word_tag_entry in word_tag.keys():
        word_tag_split = word_tag_entry.split(',')
        current_word = word_tag_split[0]
        current_tag = word_tag_split[1]
        emission_key = current_word+','+current_tag
        emission_prob[emission_key] = word_tag[word_tag_entry]/tag_count[current_tag]    
    return emission_prob

In [7]:
emission_prob = create_emission_prob_table(word_tag, tag_count)
print(emission_prob)

KeyError: ''

In [ ]:
def viterbi(trans_prob, emission_prob, tag_count, sentence):
    #initialization
    viterbi_mat = {}
    tag_sequence = []
    
    sentence_words = sentence.split()
    for i in range(0,len(sentence_words)):
        
    
    return viterbi_mat, tag_sequence